# Reading and writing CSV files, using regular expressions to validate data

In [102]:
import csv
import requests
import os
import re

We're going to work with a data file that has some item information from a serial. (Show file in Google Sheets)

Let's get the file using the python requests module. 

We will also use a method called `open()` to write a file. 

In [217]:
url = "https://go.gwu.edu/serials"
r = requests.get(url)
with open("serials-before.csv", "wb") as myfile:
    myfile.write(r.content)

In [218]:
os.listdir()

['.DS_Store',
 'pandas-options.ipynb',
 'messing_around.ipynb',
 'serials-after-cleaned.csv',
 'metadata-cleanup.ipynb',
 'lcwa_gov_pdf_metadata.csv',
 '.ipynb_checkpoints',
 'lc-python-intro',
 'serials-before.csv']

Can also go to the Jupyter Home tab and see/view the file there. 

Let's use Python to open the CSV file and look at each row:

In [221]:
with open("serials-before.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    # like a list, reader object is iterable
    for row in reader:
        #print(row)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-221-79ab9d832d84>, line 6)


**Each row of the file is a list**, so we can change it and do things with it. 

Our goal is to populate the empty "Note" field with a statement that looks like:

(write on the board)

v.VOLUME, no.ISSUE (YEAR:season).

`v.1, no.4 (1990:winter)`

Let's create a note for just for one row, to get the hang of it. Then we'll come back to the file to do the whole thing. 

**Copy and paste** one of the rows, and assign to a variable we'll call sample.

In [118]:
sample = ['99169315803604107', 'gelman', 'grcmis', '27', '4', '2016', 'Winter', '32882022255767', '']

Now, we need to put into a variable each of those elements we need to put into our note (volume, issue, etc)

In [119]:
vol = sample[3]
iss = sample[4]
year = sample[5]
season = sample[6]

In [120]:
note = "v.{}, no.{} ({}:{})".format(vol, iss, year, season.lower())
print(note)

v.27, no.4 (2016:winter)


Now we know how to create the note statement. 

We want to read each line in our CSV, formulate the note, and save our edited line so we can write it to another.

We're going to use that Accumulator pattern we learned before. As we work on a row, we'll add it to the list. 

(We're doing that so we can then write that data to a new file later.)

In [171]:
lines = []
with open("serials-before.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    # skip over the header
    next(reader)
    # like a list, reader object is iterable
    for row in reader:
        vol = row[3]
        iss = row[4]
        year = row[5]
        season = row[6]
        row[-1] = "v.{}, no.{} ({}:{})".format(vol, iss, year, season.lower())
        lines.append(row)

In [122]:
len(lines)

101

In [123]:
print(lines[0])

['99169315803604107', 'gelman', 'grcmis', '1', '1', '1989', 'Spring', '32882022255379', 'v.1, no.1 (1989:spring)']


Now we have the lines for our new file, let's **write it** to a new file. 

In [124]:
with open("serials-after.csv", 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    # create and write the header. Each line is a list:
    header = ['MMS ID', 'Library', 'Location', 'Volume', 'Issue number', 'Year', 'Month', 'Barcode', 'Note']
    writer.writerow(header)
    for line in lines:
        writer.writerow(line)

Either use !head or open it up from the Jupyter Hub home page.

In [215]:
#!head serials-after.csv

We worked with this data as if it were all OK. One thing we could do is check the data to see if there are errors in the data. Maybe we want to print a warning if there's a problem with the barcode for the item. 

Barcodes should be X characters long, all digits, and start with the same prefix each time. 

## A bit about regular expressions in python:

Use the re library and the re.compile method.

`re.compile()`

Compiles a regular expression pattern into a regular expression object which can be used to match. We want to see if all of our barcodes start with the pattern 32882.


In [128]:
bar_regex = re.compile("32882")

We can use the `match()` method on the regex object. Match by default starts at the beginning of the string. We don't need to specify that. 

In [197]:
result = prefix_re.match("32882022255460")

if result:
    print("has barcode prefix")

has barcode prefix


The `match()` method returns a match object if it works, and None if it does not exist. This allows us to use conditionals, based on whether something exists or is None. 

Are there exactly 14 digits in the barcode string?

In [207]:
numcheck = re.compile("\d{14}$")
result_num = numcheck.match("32882022255460")
if result_num:
    print("has exactly 14 digits")
else:
    print("does not have exactly 14 digits")

has exactly 14 digits


Try editing the barcode to see if it fails. 

Copy our code from before so that we can then add some checks to the existing loop. We can check whether the barcode starts with the correct numbers and that the barcode is all digits.  

In [208]:
lines = []

# need to use newline parameter on opening a csvfile
with open("serials-before.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    # skip over the header
    next(reader)
    for row in reader:
        # test that the barcode starts with 32882
        barcode = row[7]
        bar_test = re.compile("32882")
        bar_result = bar_test.match(barcode)
        if bar_result:
            vol = row[3]
            iss = row[4]
            year = row[5]
            season = row[6]
            row[-1] = "v.{}, no.{} ({}:{})".format(vol, iss, year, season.lower())
            lines.append(row)
        else:
            print("invalid barcode: {}".format(barcode))


invalid barcode: 2882022255908
invalid barcode: 2882022256401


We know from before how to write a regular expression that checks that it is only 14 digits. 

### SLIDE: 
Write a function called **check_digits()** that will test whether a barcode is all digits, and returns True if it is, and False if not. Add it to the loop.

In [201]:
def check_digits(barcode):
    numcheck = re.compile("^\d{14}$")
    result_num = numcheck.match(barcode)
    if result_num:
        return True
    else:
        return False

Once you have the function, add it to the loop. 

Hint: You only want the note to be created if both the bar_result and result of `check_digits()` are conditions that pass. 
Bonus: If you finish, make your error more informative by having the MMSID appear in the error message.


If you finish, make your error more informative by having the MMSID appear in the error message. 


In [209]:
lines = []
with open("serials-before.csv", newline="") as csvfile:
    reader = csv.reader(csvfile)
    # skip over the header
    next(reader)
    for row in reader:
        # test that the barcode starts with 32882
        barcode = row[7]
        bar_test = re.compile("32882")
        bar_result = bar_test.match(barcode)
        
        # test the length and characters in the barcode 
        num_result = check_digits(barcode)
        
        if bar_result and num_result:
            if check_digits(barcode):   
                vol = row[3]
                iss = row[4]
                year = row[5]
                season = row[6]
                row[-1] = "v.{}, no.{} ({}:{})".format(vol, iss, year, season.lower())
                lines.append(row)
        elif not bar_result:
            print("invalid barcode: {}".format(barcode))
        elif not num_result:
            print("invalid characters in barcode: {}".format(barcode))

invalid characters in barcode: 328820222563
invalid barcode: 2882022255908
invalid barcode: 2882022256401
invalid characters in barcode: 3288202225543X
invalid characters in barcode: 3288202225568X


## Write to csv

Copy our code that we used earlier to write to a file. 

In [210]:
with open("serials-after-cleaned.csv", 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    # create and write the header. Each line is a list:
    header = ['MMS ID', 'Library', 'Location', 'Volume', 'Issue number', 'Year', 'Month', 'Barcode', 'Note']
    writer.writerow(header)
    for line in lines:
        writer.writerow(line)

In [211]:
!head serials-after-cleaned.csv

Note how the CSV writer was smart and put the issue statement in quotes. This makes sure that the comma doesn't cause a problem. 

## Wrap-up

We learned to:
+ Read in a CSV
+ Use the csv libraries (also some exposure to requests)
+ Use regular expressions on real data
+ Clean the data and write out using .format()
+ Write out a CSV file at the end. 

Next steps:
+ learn about dictionaries
+ use the loc.gov JSON API

